<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Update and Manage AWS User Permissions using unSkript legos.</b>
</div>

<br>

<center><h2>Update and Manage AWS User Permissions</h2></center>

# Steps Overview
1) List all the IAM user attached policies. 
2) Attache new policy to the given user.

Here we will use unSkript AWS List Attached User Policies Lego. This lego takes UserName as input. This inputs is used to list all the policies for given user.

In [3]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from botocore.exceptions import ClientError
import pprint
from beartype import beartype


@beartype
def aws_list_attached_user_policies_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_attached_user_policies(handle, UserName: str) -> List:
    """aws_list_attached_user_policies returns the list of policies attached to the user.

        :type UserName: string
        :param UserName: IAM user whose policies need to fetched.

        :rtype: List with with the attched policy names.
    """
    result = []
    user_list = []
    ec2Client = handle.client('iam')
    try:
        response = ec2Client.list_attached_user_policies(UserName=UserName)
        for i in response["AttachedPolicies"]:
            result.append(i['PolicyName'])

    except ClientError as error:
        result.append(error.response)

    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "UserName": "UserName"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_attached_user_policies, lego_printer=aws_list_attached_user_policies_printer, hdl=hdl, args=args)

Here we will use unSkript AWS Attach IAM Policy Lego. This lego takes UserName and PolicyName as input. This inputs is used to Attach a new policy to the given user and provide specified permissions.

In [13]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from botocore.exceptions import ClientError
import pprint


from beartype import beartype
@beartype
def aws_attache_iam_policy_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_attache_iam_policy(handle, UserName: str, PolicyName: str) -> Dict:
    """aws_attache_iam_policy used to provide user permissions.

        :type handle: object
        :param handle: Object containing global params for the notebook.

        :type UserName: dict
        :param UserName: Dictionary of credentials info.

        :type PolicyName: string
        :param PolicyName: Policy name to apply the permissions to the user .

        :rtype: Dict with User policy info.
    """
    result = {}
    iamResource = handle.resource('iam')
    try:
        user = iamResource.User(UserName)
        response = user.attach_policy(
            PolicyArn='arn:aws:iam::aws:policy/'+PolicyName
            )
        result = response
    except ClientError as error:
        result = error.response

    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "PolicyName": "Policy_Name",
    "UserName": "UserName"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_attache_iam_policy, lego_printer=aws_attache_iam_policy_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS to perform AWS actions. This runbook is used to list all the attached policies to IAM user and and attach new policy to the same user. To view the full platform capabilities of unSkript please visit https://unskript.com